In [1]:
import pandas as pd
import cpi
import random
cpi.update()
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [2]:
raw_data = pd.read_csv("data/data.csv")
raw_data.drop(columns="Unnamed: 0", inplace=True)
raw_data.head(10)

,State,District,Incumbent/Challenger/Open,Receipts 18m,Contributions from Individuals 18m,Contributions from PACs and Other Committees 18m,Contributions and Loans from the Candidate 18m,Disbursements 18m,Cash On Hand,Debts 18m,Coverage End Date,Year,Candidate last name,Candidate first name,Candidate middle name,Stage,Party,Candidate votes,Total votes,Won?
0,ALABAMA,2,Incumbent,231924.0,75664.0,124850.0,0.0,233622.0,419670.0,0.0,1990-06-30 00:00:00,1990,DICKINSON,WILLIAM,LOUIS,GEN,REPUBLICAN,87649.0,170911,True
1,ALABAMA,2,Challenger,90050.0,16329.0,47300.0,16546.0,47480.0,42569.0,26535.0,1990-06-30 00:00:00,1990,BAGGIANO,FAYE,S,GEN,DEMOCRAT,83243.0,170911,False
2,ALABAMA,3,Challenger,11458.0,8824.0,100.0,2534.0,9240.0,2218.0,0.0,1990-06-30 00:00:00,1990,SLEDGE,DON,NaN,GEN,REPUBLICAN,36317.0,138242,False
3,ALABAMA,4,Incumbent,169831.0,16940.0,79650.0,0.0,116445.0,567033.0,0.0,1990-06-30 00:00:00,1990,BEVILL,TOM,NaN,GEN,DEMOCRAT,129872.0,130212,True
4,ALABAMA,5,Open,90732.0,64357.0,9800.0,11575.0,58704.0,32027.0,4400.0,1990-06-30 00:00:00,1990,MCDONALD,ALBERT,CLYDE,GEN,REPUBLICAN,55326.0,168383,False
5,ALABAMA,7,Incumbent,160773.0,48097.0,106300.0,0.0,64201.0,175205.0,0.0,1990-06-30 00:00:00,1990,HARRIS,CLAUDE,JR,GEN,DEMOCRAT,127490.0,180794,True
6,ALABAMA,7,Challenger,10829.0,6826.0,0.0,4000.0,9737.0,1091.0,4000.0,1990-06-30 00:00:00,1990,BARKER,MICHAEL,DEAN,GEN,REPUBLICAN,53258.0,180794,False
7,ALASKA,0,Challenger,96646.0,25629.0,6750.0,59547.0,86486.0,10160.0,43321.0,1990-06-30 00:00:00,1990,DEVENS,JOHN,NaN,GEN,DEMOCRAT,91677.0,191647,False
8,ARKANSAS,1,Challenger,8400.0,7840.0,500.0,0.0,7188.0,1152.0,0.0,1990-06-30 00:00:00,1990,HAYES,TERRY,OWEN,GEN,REPUBLICAN,56071.0,157097,False
9,ARKANSAS,2,Open,322126.0,255382.0,59650.0,0.0,254408.0,67718.0,4683.0,1990-06-30 00:00:00,1990,THORNTON,RAY,NaN,GEN,DEMOCRAT,103471.0,171271,True


In [3]:
# Adjust dollars to inflation
data = raw_data[[
    "Receipts 18m", 
    "Contributions from Individuals 18m", 
    "Contributions from PACs and Other Committees 18m",
    "Contributions and Loans from the Candidate 18m",
    "Disbursements 18m",
    "Cash On Hand",
    "Debts 18m",
    "Year",
    "Won?"
]]
data["Receipts 18m"] = data.apply(lambda x: cpi.inflate(x["Receipts 18m"], x["Year"]), axis=1)
data["Contributions from Individuals 18m"] = data.apply(lambda x: cpi.inflate(x["Contributions from Individuals 18m"], x["Year"]), axis=1)
data["Contributions from PACs and Other Committees 18m"] = data.apply(lambda x: cpi.inflate(x["Contributions from PACs and Other Committees 18m"], x["Year"]), axis=1)
data["Contributions and Loans from the Candidate 18m"] = data.apply(lambda x: cpi.inflate(x["Contributions and Loans from the Candidate 18m"], x["Year"]), axis=1)
data["Disbursements 18m"] = data.apply(lambda x: cpi.inflate(x["Disbursements 18m"], x["Year"]), axis=1)
data["Cash On Hand"] = data.apply(lambda x: cpi.inflate(x["Cash On Hand"], x["Year"]), axis=1)
data["Debts 18m"] = data.apply(lambda x: cpi.inflate(x["Debts 18m"], x["Year"]), axis=1)
data.head(10)

/var/folders/m6/kmzxqg9137589ftftpz5mmdc0000gn/T/ipykernel_79368/4196301287.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Receipts 18m"] = data.apply(lambda x: cpi.inflate(x["Receipts 18m"], x["Year"]), axis=1)
/var/folders/m6/kmzxqg9137589ftftpz5mmdc0000gn/T/ipykernel_79368/4196301287.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Contributions from Individuals 18m"] = data.apply(lambda x: cpi.inflate(x["Contributions from Individuals 18m"], x["Year"]), axis=1)
/var/folders/m6/kmzx

,Receipts 18m,Contributions from Individuals 18m,Contributions from PACs and Other Committees 18m,Contributions and Loans from the Candidate 18m,Disbursements 18m,Cash On Hand,Debts 18m,Year,Won?
0,480829.734353,156868.202601,258841.656465,0.000000,484350.063810,8.700687e+05,0.000000,1990,True
1,186693.561591,33853.627621,98063.358837,34303.516603,98436.538638,8.825495e+04,55012.922341,1990,False
2,23754.967559,18294.103137,207.322112,5253.542311,19156.563122,4.598404e+03,0.000000,1990,False
3,352097.215532,35120.365723,165132.061974,0.000000,241416.232976,1.175585e+06,0.000000,1990,True
4,188107.498393,133426.291431,20317.566947,23997.534430,121706.372456,6.639905e+04,9122.172915,1990,False
5,333317.978653,99715.716067,220383.404744,0.000000,133102.868936,3.632387e+05,0.000000,1990,True
6,22450.911477,14151.807345,0.000000,8292.884468,20186.954017,2.261884e+03,8292.884468,1990,False
7,200368.528080,53134.584009,13994.242540,123454.097858,179304.601530,2.106393e+04,89814.012012,1990,False
8,17415.057383,16254.053558,1036.610559,0.000000,14902.313389,2.388351e+03,0.000000,1990,False
9,667838.425555,529463.355318,123667.639633,0.000000,527444.037950,1.403944e+05,9708.894491,1990,True


In [14]:
# Split into training and test data:
TRAIN_SPLIT_FACTOR = 0.75
TEST_SPLIT_FACTOR = 1 - TRAIN_SPLIT_FACTOR
num_samples = data.shape[0]
num_train_samples = int(TRAIN_SPLIT_FACTOR * num_samples)
train_indices = random.sample(range(num_samples), num_train_samples)
test_indices = list(set(range(num_samples)) - set(train_indices))
assert len(train_indices) + len(test_indices) == num_samples

train_data_x = data.iloc[train_indices, 0:-2]
train_data_y = data.iloc[train_indices, -1]
test_data_x = data.iloc[test_indices, 0:-2]
test_data_y = data.iloc[test_indices, -1]

In [15]:
train_data_x = scaler.fit_transform(train_data_x.to_numpy())
test_data_x = scaler.fit_transform(test_data_x.to_numpy())
train_data_y = train_data_y.to_numpy()
test_data_y = test_data_y.to_numpy()
print(train_data_x)
print(train_data_x.shape)
print(test_data_y.shape)

[[ 1.98122911  0.92131022  3.32095334 ...  1.27183748  2.05312842
  -0.24439347]
 [-0.76525972 -0.61623414 -0.78850026 ... -0.66508516 -0.67291727
  -0.09041517]
 [-0.07664766 -0.23865696  0.27703935 ... -0.09961319 -0.25398339
   2.02501564]
 ...
 [-0.01515297 -0.3358685  -0.15814476 ...  0.29183627 -0.51248253
   0.98089382]
 [-0.59185579 -0.50971111 -0.52704571 ... -0.62852837  0.07930994
  -0.24439347]
 [-0.74968163 -0.73935614 -0.57314057 ... -0.11576663 -0.34280217
  -0.24439347]]
(3586, 7)
(1196,)


In [16]:
model = LogisticRegression().fit(train_data_x, train_data_y)

In [19]:
predictions = model.predict(test_data_x)

In [21]:
total_predictions = len(predictions)
accuracy = (predictions == test_data_y).sum() / total_predictions
print(accuracy)

0.8578595317725752
